# Getting Start

### Installation of JanusQ
JanusQ can be installed by docker, wheel, and soruce code. Docker is recommended, as all functions have been tested on it.
Linux (Ubuntu 22.04 latest) platform and Python (3.10) is prefered when instaling by wheel or soruce code.

**From docker.**
Pull docker using docker 
        
        docker pull jxhhhh/janusq:latest

The code is in "/home/JanusQ-main". The examples that can be directly run is in "/home/JanusQ-main/examples"

**From wheel.**
Download janusq.whl from "JanusQ-main/dist".

        pip install janusq.whl

**From source code.**
Run following commends.

        git clone git@github.com:JanusQ/JanusQ.git
        cd JanusQ
        pip install -r requirements.txt

### Structure of JanusQ

- JanusCT
  - vectorization
  - fidelity preidiction
  - fidelity optimization
  - unitary decomposition
  - bug identification
- JanusFEM
  - benchmarking.py. Generate a circuit for measuring calibration matrices.
  - mitigation.py. Implement iterative calibration, which includes quantifying interactions between qubits and constructing Bayesian networks.
  - tools.py. Implement data format conversion.
- HyQSAT
  - solve sat domain problem
- time crystal

### Note
1. Permission denied 'minisat_core'  
chmod +x janusq/hyqsat/minisat_core

### Test JanusQ

In [1]:
import sys
sys.path.append('..')

import logging
logging.basicConfig(level=logging.ERROR)

#### Test Janus-CT

In [2]:
# test janus-ct
from janusq.data_objects.backend import LinearBackend
from janusq.data_objects.random_circuit import random_circuits
from janusq.analysis.vectorization import RandomwalkModel
# define a 10-qubit backend
n_qubits = 8
n_steps = 1
n_walks = 20
backend = LinearBackend(n_qubits, 1)

circuits = random_circuits(backend, n_circuits=300, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

vec_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend)
vec_model.train(circuits, multi_process=False, remove_redundancy = False)

100%|██████████| 300/300 [00:02<00:00, 136.59it/s]


#### Test Janus-FEM

In [3]:
# test readout calibration
from janusq.simulator.noisy_simulator import NoisySimulator
from janusq.data_objects.backend import LinearBackend
from janusq.simulator.readout_error_model import ReadoutErrorModel
from janusq.optimizations.readout_mitigation.fem import  EnumeratedProtocol
from janusq.data_objects.algorithms import get_algorithm_circuits

n_qubits = 3
backend = LinearBackend(n_qubits, 1)
circuit = get_algorithm_circuits(n_qubits, backend, algs = ['ghz'])[0]

samples = 10000

simulator = NoisySimulator(
    backend, readout_error_model = ReadoutErrorModel.random_model(backend))
output_noise = simulator.execute(circuit,samples)


protocol = EnumeratedProtocol(n_qubits)
real_bstrs, circuits_protocol = protocol.gen_circuits()

all_statuscnts = [
    simulator.execute(cir_protocol, samples)
    for cir_protocol in circuits_protocol
]

print(all_statuscnts)


[{'110': 2, '101': 10, '011': 14, '100': 81, '001': 701, '010': 245, '000': 8947}, {'100': 1, '010': 5, '000': 118, '101': 91, '111': 4, '011': 222, '001': 9559}, {'110': 2, '101': 11, '011': 26, '100': 98, '001': 707, '010': 256, '000': 8900}, {'100': 3, '110': 86, '001': 44, '011': 612, '111': 12, '000': 566, '010': 8677}, {'110': 2, '101': 7, '000': 2, '010': 92, '001': 660, '111': 84, '011': 9153}, {'101': 1, '100': 4, '001': 43, '111': 6, '011': 636, '110': 75, '000': 581, '010': 8654}, {'101': 4, '110': 3, '011': 14, '100': 75, '001': 710, '010': 218, '000': 8976}, {'100': 2, '111': 1, '011': 259, '101': 99, '010': 4, '000': 118, '001': 9517}, {'101': 7, '110': 2, '011': 13, '100': 79, '001': 655, '010': 245, '000': 8999}, {'001': 56, '011': 1, '111': 15, '110': 193, '010': 15, '000': 885, '101': 657, '100': 8178}, {'010': 1, '000': 11, '100': 90, '001': 912, '011': 19, '111': 221, '101': 8746}, {'011': 1, '111': 17, '001': 62, '010': 24, '000': 880, '110': 199, '101': 607, '100'

#### Test Janus-SAT

In [ ]:
# test hyqsat calibration
# input cnf flie
file_path = "cnf_examples/uf50-01.cnf"
from janusq.hyqsat import readCNF

print(readCNF(file_path))